In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, box

In [ ]:
desired_regions = snakemake.params.aggregated_regions

In [ ]:
geodata_files = {
    "onshore": snakemake.input.euroshape,
    "offshore_bottom": snakemake.input.eurooffshoreshape,
}

In [ ]:
offshore_to_ISO3166 = {
    "Albania"        : "AL",
    "Belgium"        : "BE",
    "Bulgaria"       : "BG",
    "Croatia"        : "HR",
    "Cyprus"         : "CY",
    "Denmark"        : "DK",
    "Estonia"        : "EE",
    "Finland"        : "FI",
    "France"         : "FR",
    "Germany"        : "DE",
    "Greece"         : "GR",
    "Ireland"        : "IE",
    "Italy"          : "IT",
    "Latvia"         : "LV",
    "Lithuania"      : "LT",
    "Malta"          : "MT",
    "Netherlands"    : "NL",
    "Poland"         : "PL",
    "Portugal"       : "PT",
    "Romania"        : "RO",
    "Spain"          : "ES",
    "Slovenia"       : "SI",
    "Slovakia"       : "SK",
    "Sweden"         : "SE",
    "United Kingdom" : "UK"
}

In [ ]:
# Onshore shape

In [ ]:
if snakemake.wildcards.spatial== "region":
    nuts_lvl=0    
if snakemake.wildcards.spatial== "nuts2":
    nuts_lvl=2

euro_onshore = (
        gpd.read_file(snakemake.input.euroshape)
        .replace({"GB": "UK", "EL": "GR"})
        .query("LEVL_CODE == @nuts_lvl & CNTR_CODE in @desired_regions ")
        .rename(columns={"NUTS_ID": "index"})
        .loc[:,["index","CNTR_CODE","geometry"]]
        .set_index(["index"])
    )

In [ ]:
rectx1 = -12
rectx2 = 44
recty1 = 33
recty2 = 81


polygon = Polygon(
    [
        (rectx1, recty1),
        (rectx1, recty2),
        (rectx2, recty2),
        (rectx2, recty1),
        (rectx1, recty1),
    ]
)

euro_onshore=gpd.clip(euro_onshore,polygon)

euro_onshore.plot()

In [ ]:
# Remove Svalbard

euro_onshore=gpd.clip(euro_onshore,box(rectx1,recty1,rectx2,72))

euro_onshore.plot()

In [ ]:
# Remove Jan Mayen

nor=(euro_onshore.query("CNTR_CODE =='NO'")
         .clip(box(0,recty1,rectx2,recty2)))

euro_onshore=pd.concat([euro_onshore.query("CNTR_CODE != 'NO'"),nor])

euro_onshore.plot()

In [ ]:
euro_onshore.to_file(snakemake.output.onshoreshape)

In [ ]:
# Offshore shape

In [ ]:
# euro_offshore = (
#     gpd.read_file(snakemke.input.eurooffshoreshape)
#     .replace(offshore_to_ISO3166)
#     .set_index("index")
#     .filter(items=desired_regions,axis=0)
# )

euro_offshore = (
     gpd.read_file(snakemake.input.eurooffshoreshape)
     .replace(offshore_to_ISO3166))
#     .set_index("index")
#     .filter(items=desired_regions,axis=0)
# )

euro_offshore=gpd.clip(euro_offshore,polygon)

euro_offshore.plot()

In [ ]:
norway=(euro_offshore.query("~NUTS_ID.isna()",engine="python")
        .dissolve()
        .assign(index="NO"))

euro_offshore=pd.concat(
        [euro_offshore.query("NUTS_ID.isna()",engine="python"),
         norway])

In [ ]:
# Remove Svalbard

euro_offshore=gpd.clip(euro_offshore,box(rectx1,recty1,rectx2,72))

euro_offshore.plot()

In [ ]:
(euro_offshore.set_index("index")
        .filter(items=desired_regions,axis=0)
        .to_file(snakemake.output.offshoreshape))